In this study, I present a comprehensive pipeline for training a machine learning model specifically designed for a long/short trading strategy. Central to this approach is the development of a customized loss function aimed at optimizing trading outcomes.

We begin by categorizing returns into 10 deciles based on individual stock performance. While this method, influenced by the weak-form Efficient Market Hypothesis (EMH), may not yield significant results in isolation, it serves as a foundational step.

In practical applications, cross-sectional return deciles or rankings are often employed to extract more nuanced information for trading. Trading on a diversified portfolio of stocks is crucial to leverage the expectations derived from probabilities, rather than relying solely on predictions for individual stocks. However, this study emphasizes the pipeline for implementing machine learning in finance, so we will not delve into portfolio diversification here.

Further research and detailed strategy backtesting will be available in the "Strategy Backtesting" repository on my GitHub.

In this study, I utilize technical indicators to validate the model's ability to uncover valuable insights. For real-world applications, it is imperative to incorporate a broader range of data, including macroeconomic indicators, microeconomic factors, company characteristics, text sentiment analysis, and behavioral data. This comprehensive approach ensures a robust and well-informed trading strategy.

This study underscores the potential of machine learning in quantitative finance, demonstrating how these advanced techniques can revolutionize trading strategies.

# Outline:
- Load data from Yahoo Finance & Get some technical indicators
- Custom Loss Function
- Implement Gaussian Naive Bayes
- RandomizedSearchCV on Random Forest


In [ ]:
!pip install yfinance
!pip install pandas_ta

In [ ]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

###Load data from Yahoo Finance & Get some technical indicators

In [ ]:
df= pd.DataFrame()
df=df.ta.ticker('pfe')

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [ ]:
# Exponential Moving Averages
df['ema10']=ta.ema(df['Close'],length=10)
df['ema30']=ta.ema(df['Close'],length=30)

# Average True Range- Measures Volatility Caused by Price Gaps or Limit Moves
# true range = max[(high - low), abs(high - previous close), abs (low - previous close)]
df['atr'] = ta.atr(df['High'],df['Low'],df['Close']) # # need to be normalized by price

# Average Directional Movement Index - to  quantify trend strength by measuring
# the amount of movement in a single direction

adx= ta.adx(df['High'],df['Low'],df['Close'])
df['adx'] = adx['ADX_14']

######

# Moving Average Convergence/ Divergence
#   Used to identify aspects of a security's overall trend
#   MACD Line: (12-day EMA - 26-day EMA)
#   Signal Line: 9-day EMA of MACD Line
#   MACD Histogram: MACD Line - Signal Line

macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
df['macd']=macd['MACD_12_26_9']/df['Close'] #normalize by current price to make the indicator stationary
df['macds']=macd['MACDs_12_26_9']/df['Close'] #normalize by current price to make the indicator stationary


# Relative Strength Index
#   momentum oscillator used to measure the
#   velocity as well as the magnitude of directional price movements

df['rsi'] =ta.rsi(df['Close'],length=14)

#df['Cgtema10'] = np.where(df['Close'] > df['ema10'], 1, -1) # generate categorical variable to indentify short term trend
#df['ema10gtema30'] = np.where(df['ema10'] > df['ema30'], 1, -1) # generate categorical variable to indentify mid-short term trend
#df['macdsgtmacd'] = np.where(df['macd'] > df['macds'] , 1, -1) # generate categorical variable to indentify trend of trend
df['Return_1'] = df['Close'].pct_change(1).shift(-1)


In [ ]:
# Calculate the decile thresholds using the specified date range
quantiles = 10  # or any number like 15 for 15 quantiles
thresholds = pd.qcut(df.loc['1981-01-01':'2020-12-31']['Return_1'], quantiles, retbins=True)[1]

# Classify each return based on the decile thresholds
df['target'] = pd.cut(df['Return_1'], bins=thresholds, labels=False, include_lowest=True)
df.dropna(inplace=True)
df['target'] = df['target'].astype(int)


# Features
predictors_list = ['atr', 'adx','rsi', 'macd', 'macds']
X = df[predictors_list]

# Target Variable
y = df.target

In [ ]:
X_train=X.loc['1981-01-01':'2020-12-31']
X_valid=X.loc['2021-01-01':'2022-12-31']
X_test=X.loc['2023-01-01':]
y_train=y.loc['1981-01-01':'2020-12-31']
y_valid=y.loc['2021-01-01':'2022-12-31']
y_test=y.loc['2023-01-01':]

X_train_valid = pd.concat((X_train, X_valid), axis = 0)
y_train_valid = pd.concat((y_train, y_valid), axis = 0)

In [ ]:
print (X_train.shape)
X_train.head()

(10087, 5)


,atr,adx,rsi,macd,macds
Date,,,,,
1981-01-02 00:00:00-05:00,0.006425,25.968779,72.849067,0.032549,0.021637
1981-01-05 00:00:00-05:00,0.006332,27.632991,69.144350,0.033372,0.024148
1981-01-06 00:00:00-05:00,0.006934,29.528765,72.190342,0.034257,0.025811
1981-01-07 00:00:00-05:00,0.006942,31.289127,68.546804,0.034627,0.027768
1981-01-08 00:00:00-05:00,0.006950,33.097560,62.593050,0.033510,0.029287


###Define custom loss function:


In this application, I focus on "probability" rather than prediction. When implementing a long-short strategy, the goal is to long a selection of stocks with a high probability of going up and short a selection of stocks with a high probability of going down.

In other words, we do not bet on individual stocks. Instead, we rely on the law of expectation. We aim to profit based on "expectation" by exploiting the predicted probabilities. If, on average, the stocks we predict have a higher probability of going up compared to others, and similarly for going down, we can generate profit by creating cross-sectional signals and utilizing a long-short strategy.

In this document, I illustrate how to utilize a machine learning model to generate insights or predicted probabilities. For demonstration purposes, I apply the methodology to only one stock.

Misclassifications should have varying penalties. A prediction of "up" that results in a "down" movement should incur a higher penalty. Similarly, a prediction of "down" that results in an "up" movement should also incur a higher penalty.

Custom Loss Function: This function captures the ordinal relationship by computing the custom loss using predicted probabilities and a penalty matrix. For each instance, it retrieves the true class and the predicted probabilities, then computes the penalty by performing a dot product between the predicted probabilities and the penalties associated with the true class. The total loss is accumulated across all instances.


In [ ]:
def create_penalty_matrix(num_classes):
    penalty_matrix = np.zeros((num_classes, num_classes))
    for i in range(num_classes):
        for j in range(num_classes):
            penalty_matrix[i, j] = abs(i - j)
    return penalty_matrix

def custom_loss(y_true, y_pred_proba, num_classes):
    # Define custom penalty matrix
    penalty_matrix = create_penalty_matrix(num_classes)
    """ # for example: if there are 3 classes, the penalty matrix would be:
    np.array([
        [0, 1, 2],  # Penalty for predicting 'up' as 'same' or 'down'
        [1, 0, 1],  # Penalty for predicting 'same' as 'up' or 'down'
        [2, 1, 0]   # Penalty for predicting 'down' as 'same' or 'up'
    ])
    """
    total_loss = 0
    for i in range(len(y_true)):
        true_class = y_true[i]
        predicted_probabilities = y_pred_proba[i]
        penalties = penalty_matrix[true_class]
        loss = np.dot(predicted_probabilities, penalties)
        total_loss += loss
    return total_loss/ len(y_true) # average loss for each prediction

### Implement Gaussian Naive Bayes


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, confusion_matrix

NB doc:
https://scikit-learn.org/stable/modules/naive_bayes.html

GaussianNB doc:
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

**priors:** array-like of shape (n_classes,) \
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.

**var_smoothing:** float, default=1e-9 \
Portion of the largest variance of all features that is added to variances for calculation stability.



In [ ]:
# Assuming X_train, y_train, X_valid, y_valid are already defined
for vs in np.logspace(0, -12, num=13):
    # Initialize Gaussian Naive Bayes
    gnb = GaussianNB(var_smoothing=vs)
    # Train the classifier
    gnb.fit(X_train, y_train)

    # Predict probabilities on training and validation data
    y_pred_proba_train = gnb.predict_proba(X_train)
    y_pred_proba_valid = gnb.predict_proba(X_valid)

    # Calculate custom loss for training and validation sets
    loss_train = custom_loss(y_train, y_pred_proba_train, quantiles )
    loss_valid = custom_loss(y_valid, y_pred_proba_valid, quantiles )

    print('vs = ' + str(vs))
    print('Custom Loss (Training) = ' + str(loss_train))
    print('Custom Loss (Validation) = ' + str(loss_valid))

vs = 1.0
Custom Loss (Training) = 3.2932322031102785
Custom Loss (Validation) = 3.329622023519707
vs = 0.1
Custom Loss (Training) = 3.2929334727001343
Custom Loss (Validation) = 3.340533090865986
vs = 0.01
Custom Loss (Training) = 3.2909695293967176
Custom Loss (Validation) = 3.347406642185016
vs = 0.001
Custom Loss (Training) = 3.276277247128503
Custom Loss (Validation) = 3.4454284551934444
vs = 0.0001
Custom Loss (Training) = 3.2683478201362215
Custom Loss (Validation) = 3.9155602238241047
vs = 1e-05
Custom Loss (Training) = 3.2524982088472396
Custom Loss (Validation) = 4.01634883797061
vs = 1e-06
Custom Loss (Training) = 3.228002889767618
Custom Loss (Validation) = 4.011306723692208
vs = 1e-07
Custom Loss (Training) = 3.237693360562851
Custom Loss (Validation) = 4.021463302089587
vs = 1e-08
Custom Loss (Training) = 3.2403093067164734
Custom Loss (Validation) = 4.023757852428698
vs = 1e-09
Custom Loss (Training) = 3.240599960348991
Custom Loss (Validation) = 4.024009082575971
vs = 1e

In [ ]:
# print the prior for classes:
pd.DataFrame(gnb.class_prior_, index=gnb.classes_, columns=['Prior'])

,Prior
0,0.100030
1,0.100030
2,0.099931
3,0.100030
4,0.116090
5,0.083870
6,0.100030
7,0.099931
8,0.100030
9,0.100030


In [ ]:
#Predicting the probability of each class
gnb = GaussianNB(var_smoothing = 0.01)# we choose 0.01 because it got the best validation performance
gnb.fit(X_train_valid, y_train_valid)
y_pred = gnb.predict(X_test)

print (gnb.predict_proba(X_test))

[[0.09824072 0.10222692 0.09794135 ... 0.10614313 0.09590007 0.0968529 ]
 [0.10539192 0.09800141 0.09940563 ... 0.10270393 0.10200856 0.10914623]
 [0.10875905 0.09595327 0.10047349 ... 0.10079714 0.10438451 0.11453249]
 ...
 [0.1094517  0.0927042  0.10529225 ... 0.09649877 0.10955619 0.12109346]
 [0.10008538 0.09724779 0.10469472 ... 0.10091109 0.10503142 0.10891925]
 [0.0945261  0.10003313 0.10437232 ... 0.10273601 0.10224964 0.10161667]]


In [ ]:
# Calculate the loss on testing data
custom_loss(y_test, gnb.predict_proba(X_test), quantiles)

3.2845111121037767

Note: In this study, I use only technical indicators as input to demonstrate that the model can effectively learn and extract information, even with imbalanced labels. This is evidenced by the fluctuating probabilities for each class and the variations in validation loss across different hyperparameter settings.

By incorporating more meaningful inputs such as macroeconomic indicators, microeconomic factors, company characteristics, text sentiment analysis, and behavioral data, the model's performance can be significantly enhanced.

# GridSearchCV with Random Forest

Note: The RandomForestClassifier only supports Gini Impurity, Entropy, and Log Loss as criteria and does not support custom criteria. Therefore, we cannot incorporate the ordinal relationship into the splitting criteria. Our custom loss can only be applied to measure the performance.

RandomForestClassifier doc:
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

# Create custom scoring function
def custom_scorer(y_true, y_pred_proba):
    num_classes = len(np.unique(y_true))
    return custom_loss(y_true, y_pred_proba, num_classes)

# Wrap the custom scorer using make_scorer
scorer = make_scorer(custom_scorer, needs_proba=True, greater_is_better=False)

# Define refined parameter dist
param_dist = {
    'n_estimators': [10, 20, 30],  # Number of trees in random forest
    'max_features': ['sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [5, 10, 20],  # Maximum number of levels in tree
    'min_samples_split': [5, 10, 25],  # Minimum number of samples required to split a node
    'min_samples_leaf': [5, 10, 20],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Initialize RandomizedSearchCV with the custom scorer
rf = RandomForestClassifier()
clf = RandomizedSearchCV(rf, param_dist, n_iter=50, cv=3, scoring=scorer, n_jobs=-1, random_state=42, verbose=2)

# Fit the classifier to the training data
clf.fit(X_train_valid, y_train_valid)

# Get optimal parameters
print(clf.best_params_)

# Predict probabilities using the test set
y_pred_proba_test = clf.predict_proba(X_test)
y_pred_proba_train_valid = clf.predict_proba(X_train_valid)

# Calculate custom loss
num_classes = len(np.unique(y_train_valid))
test_loss = custom_loss(y_test, y_pred_proba_test, num_classes)
train_valid_loss = custom_loss(y_train_valid, y_pred_proba_train_valid , num_classes)

print('Train_Valid custom loss = ' + str(train_valid_loss))
print('Test custom loss = ' + str(test_loss))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
{'n_estimators': 10, 'min_samples_split': 25, 'min_samples_leaf': 20, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}
Train_Valid custom loss = 3.1194668720472385
Test custom loss = 3.404193646019533


In [ ]:
y_pred_proba_test

array([[0.12482348, 0.07754984, 0.10764849, ..., 0.11320992, 0.10661581,
        0.0892224 ],
       [0.14913044, 0.10958457, 0.10807601, ..., 0.07992078, 0.12453649,
        0.09250517],
       [0.09884315, 0.12067661, 0.09182381, ..., 0.07990958, 0.15068746,
        0.10501566],
       ...,
       [0.13072927, 0.07620397, 0.04691623, ..., 0.12032076, 0.17562662,
        0.10354213],
       [0.15597213, 0.04613602, 0.04127524, ..., 0.15909731, 0.12931347,
        0.11667815],
       [0.16324486, 0.04552996, 0.04430554, ..., 0.15182458, 0.10734377,
        0.10501149]])

Based on our analysis, it is evident that the Random Forest Classifier from scikit-learn does not adequately meet our requirements. To achieve better performance, we should consider utilizing advanced models such as XGBoost and Neural Networks, which are capable of handling ordinal variables more effectively. These models provide enhanced criteria for dealing with ordinal data, thereby offering a more robust solution to our problem.